# Sentinel 2 Atmospheric Correction in Google Earth Engine

### Import modules 
and initialize Earth Engine

In [1]:
import ee
from Py6S import *
import datetime
import math
import os
import sys
import numpy as np
sys.path.append(os.path.join(os.path.dirname(os.getcwd()),'bin'))
from atmospheric import Atmospheric
from AtcorFunctions import *

ee.Initialize()

### time and place
Define the time and place that you are looking for.

In [9]:
# start and end of time series
start = ee.Date('2018-06-01')
finish =ee.Date('2018-08-31')

#coordinates below need adjusting to actual site coordinates - these are just close
geom = ee.Geometry.Point([8.1191,60.0676]) 
aoi = geom.buffer(10000).bounds() #buffer is in meters, can adjust (10km buffer here)

# Whole park
#geom = ee.Geometry.Rectangle(8.3018, 60.3967,6.7596,59.9)

### an image
The following code will grab the first scene that occurs on or after date.

In [10]:
collection = ee.ImageCollection("COPERNICUS/S2")\
    .filterBounds(geom)\
    .filterDate(start,finish)\
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 80)
    #.map(s2mask())

# The first image in the collection
S2 = ee.Image(collection.first())

print(ee.Date(S2.get('system:time_start')).format('yyyy-M-d').getInfo())
print(ee.Date(S2.get('system:time_start')).format('yyyy-mm-dd').getInfo())

# top of atmosphere reflectance
toa = S2.divide(10000)

#ui.eprint(collection.size())

2018-6-3
2018-00-03


In [4]:
date=start
## METADATA
info = S2.getInfo()['properties']
scene_date = datetime.datetime.utcfromtimestamp(info['system:time_start']/1000)# i.e. Python uses seconds, EE uses milliseconds
solar_z = info['MEAN_SOLAR_ZENITH_ANGLE']

## ATMOSPHERIC CONSTITUENTS
h2o = Atmospheric.water(geom,date).getInfo()
o3 = Atmospheric.ozone(geom,date).getInfo()
aot = Atmospheric.aerosol(geom,date).getInfo()

## TARGET ALTITUDE
SRTM = ee.Image('USGS/GMTED2010')# Shuttle Radar Topography mission covers *most* of the Earth
alt = SRTM.reduceRegion(reducer = ee.Reducer.mean(),geometry = geom.centroid()).get('be75').getInfo()
km = alt/1000 # i.e. Py6S uses units of kilometers

### 6S object

The backbone of Py6S is the 6S (i.e. SixS) class. It allows you to define the various input parameters, to run the radiative transfer code and to access the outputs which are required to convert radiance to surface reflectance.

In [5]:
# Instantiate
s = SixS()

# Atmospheric constituents
s.atmos_profile = AtmosProfile.UserWaterAndOzone(h2o,o3)
s.aero_profile = AeroProfile.Continental
s.aot550 = aot

# Earth-Sun-satellite geometry
s.geometry = Geometry.User()
s.geometry.view_z = 0               # always NADIR (I think..)
s.geometry.solar_z = solar_z        # solar zenith angle
s.geometry.month = scene_date.month # month and day used for Earth-Sun distance
s.geometry.day = scene_date.day     # month and day used for Earth-Sun distance
s.altitudes.set_sensor_satellite_level()
s.altitudes.set_target_custom_altitude(km)

### Atmospheric Correction

In [6]:
def spectralResponseFunction(bandname):
    """
    Extract spectral response function for given band name
    """
    bandSelect = {
        'B1':PredefinedWavelengths.S2A_MSI_01,
        'B2':PredefinedWavelengths.S2A_MSI_02,
        'B3':PredefinedWavelengths.S2A_MSI_03,
        'B4':PredefinedWavelengths.S2A_MSI_04,
        'B5':PredefinedWavelengths.S2A_MSI_05,
        'B6':PredefinedWavelengths.S2A_MSI_06,
        'B7':PredefinedWavelengths.S2A_MSI_07,
        'B8':PredefinedWavelengths.S2A_MSI_08,
        'B8A':PredefinedWavelengths.S2A_MSI_09,
        'B9':PredefinedWavelengths.S2A_MSI_10,
        'B10':PredefinedWavelengths.S2A_MSI_11,
        'B11':PredefinedWavelengths.S2A_MSI_12,
        'B12':PredefinedWavelengths.S2A_MSI_13,
        }
    return Wavelength(bandSelect[bandname])
def toa_to_rad(bandname):
    """
    Converts top of atmosphere reflectance to at-sensor radiance
    """
    # solar exoatmospheric spectral irradiance
    ESUN = info['SOLAR_IRRADIANCE_'+bandname]
    solar_angle_correction = math.cos(math.radians(solar_z))
    # Earth-Sun distance (from day of year)
    doy = scene_date.timetuple().tm_yday
    d = 1 - 0.01672 * math.cos(0.9856 * (doy-4))# http://physics.stackexchange.com/questions/177949/earth-sun-distance-on-a-given-day-of-the-year
    # conversion factor
    multiplier = ESUN*solar_angle_correction/(math.pi*d**2)
    # at-sensor radiance
    rad = toa.select(bandname).multiply(multiplier)
    return rad
def surface_reflectance(bandname):
    """
    Calculate surface reflectance from at-sensor radiance given waveband name
    """
    # run 6S for this waveband
    s.wavelength = spectralResponseFunction(bandname)
    s.run()
    # extract 6S outputs
    Edir = s.outputs.direct_solar_irradiance             #direct solar irradiance
    Edif = s.outputs.diffuse_solar_irradiance            #diffuse solar irradiance
    Lp   = s.outputs.atmospheric_intrinsic_radiance      #path radiance
    absorb  = s.outputs.trans['global_gas'].upward       #absorption transmissivity
    scatter = s.outputs.trans['total_scattering'].upward #scattering transmissivity
    tau2 = absorb*scatter                                #total transmissivity
    # radiance to surface reflectance
    rad = toa_to_rad(bandname)
    ref = rad.subtract(Lp).multiply(math.pi).divide(tau2*(Edir+Edif))
    return ref

# # all wavebands
def imageatcorrector(eeimage):
    corrected = eeimage.select('QA60')
    for band in ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12']:
        corrected = corrected.addBands(surface_reflectance(band))
    return corrected

In [11]:
# surface reflectance rgb
#b = surface_reflectance('B2')
#g = surface_reflectance('B3')
#r = surface_reflectance('B4')
#ref = r.addBands(g).addBands(b)

# # all wavebands
output = imageatcorrector(S2)

### Display results

In [28]:
from geetools import ui, cloud_mask

collection = ee.ImageCollection("COPERNICUS/S2")\
    .filterBounds(geom)\
    .filterDate(start,finish)\
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 80)\
    .map(cloud_mask.sentinel2())
    #.map(s2mask())

# The first image in the collection
S2 = ee.Image(collection.first()).divide(10000)
output = imageatcorrector(S2)
#output2 = imageatcorrector(S2)

from IPython.display import display, Image

region = geom.buffer(10000).bounds().getInfo()['coordinates']
channels = ['B4','B3','B2']

original = Image(url=S2.select(channels).getThumbUrl({
                'region':region,'min':0,'max':0.25
                }))

corrected = Image(url=output.select(channels).getThumbUrl({
                'region':region,'min':0,'max':0.25
                }))

display(original, corrected)#, corrected2)

## Cloud mask functions (from *geetools*)

In [85]:
# Import relevant functions
sys.path.append(os.path.join(os.path.dirname(os.getcwd()),'geetools'))
from geetools import ui, cloud_mask

In [60]:
MapS2 = ui.Map(tabs=('Inspector',))
MapS2.show()

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [61]:
#visS2 = {'bands':['B4','B3','B2'],'min':0, 'max':5000}
visS2 = {min: 0.0,max: 0.25,'bands':channels}
is2=output#S2/output
is2=is2.clip(aoi)

MapS2.centerObject(is2, zoom=11)
MapS2.addLayer(is2,visS2, 'Sentinel 2 Original')

### ESA Cloud mask

In [ ]:
ESA_mask_all = cloud_mask.sentinel2()
is2_ESA = ESA_mask_all(is2)
MapS2.addLayer(is2_ESA, visS2, 'Sentinel 2 ESA maked')

In [34]:
p = geom
mission = 'COPERNICUS/S2'
first= ee.Date('2018-08-01')
last = ee.Date('2018-08-31')

#atcorrection=imageatcorrector()
colnoatcor = ee.ImageCollection(mission)\
    .filterBounds(p).filterDate(first,last)\
    .map(cloud_mask.sentinel2())\
    .map(indices.ndvi('B8','B4'))

ColList = colnoatcor.toList(colnoatcor.size()) # must loop through lists
NO_OF_IMAGES = ColList.size().getInfo()  # no. of images in the collection
NO_OF_IMAGES

12

### Mapping atmospheric correction over an image collection

In [39]:
# Have to iterate through an ee.List, so must convert image collection to this format
S4 = ColList
CorList = ee.List([0]) # Can't init empty list so need a garbage element
export_list = []
coeff_list = []
for i in range(NO_OF_IMAGES):
    img = imageatcorrector(ee.Image(S4.get(i)))
    CorList = CorList.add(img)

CorList = CorList.slice(1) #removing that garbage element

In [62]:
CorList.size().getInfo()==S4.size().getInfo() #check that the lists are equal sizes *all images corrected*
CorCol = ee.ImageCollection(CorList) #convert list back to an image collection

### Composites

In [63]:
from geetools import ui, tools, composite, cloud_mask, indices

In [64]:
bands = ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12']

In [65]:
medoid = composite.medoid(CorCol, bands=bands)

#### Show on map

In [69]:
MapS2.addLayer(medoid.clip(aoi), visS2, 'Medoid AtCorrected')

In [72]:
Map = ui.Map()
Map.show()
vis = {'bands':['B4', 'B3','B2'], 'min':0, 'max':5000}
Map.addLayer(p)
Map.centerObject(p)
#Map.addLayer(max_ndvi, vis, 'max NDVI')
#Map.addLayer(mosaic, vis, 'simply Mosaic')
Map.addLayer(medoid, vis, 'Medoid AtCorrected')
Map.addLayer(medoid0, vis, 'Medoid Not AtCorrected')

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

### Sentinel Hub Cloud mask (*uses machine learning*)

In [ ]:
import s2cloudless
#s2cloudless.test_sentinelhub_cloud_detector() 
from s2cloudless import S2PixelCloudDetector, CloudMaskRequest

In [ ]:
cloud_detector = S2PixelCloudDetector(threshold=0.4, average_over=4, dilation_size=2)
cloud_probs = cloud_detector.get_cloud_probability_maps(np.array(S2))
#cloud_masks = cloud_detector.get_cloud_masks(np.array(wcsbands))

### Export to Asset

In [ ]:
# # set some properties for export
dateString = scene_date.strftime("%Y-%m-%d")
ref = ref.set({'satellite':'Sentinel 2',
              'fileID':info['system:index'],
              'date':dateString,
              'aerosol_optical_thickness':aot,
              'water_vapour':h2o,
              'ozone':o3})

In [ ]:
# define YOUR assetID 
assetID = 'users/visithuruvixen/test'

In [ ]:
# # export
export = ee.batch.Export.image.toAsset(\
    image=output,
    description='sentinel2_atmcorr_export',
    assetId = assetID,
    region = region,
    scale = 30)

# # uncomment to run the export
export.start() 